## TP7


* Implementar el detector de fondo naive usando la mediana como
estimador El algoritmo debe recibir el parámetro N (cantidad de
frames utilizados para la estimación) y el intervalo de tiempo para
recalcular el fondo
* Se deben generar las mascaras de foreground y aplicarlas a los frames
para segmentar los objetos en movimiento
* Comparar con alguno de los métodos vistos en la practica basados en
mezcla de gaussiana s


In [40]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib inline

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [41]:
def takeFirstNFramesMedian(filename, N):
    frame_list = []
    while True:
        cap = cv.VideoCapture(filename)

        ret, frame = cap.read()

        frame_list.append(frame)

        cv.waitKey(1)
        if len(frame_list) == N:
            cap.release()
            return frame_list

class NaiveBackgroundSubs:
    
    def __init__(self, N, interval):
        self.median_gray = 0
        self.last_frames = [] # acumulo frames anteriores
        self.N = N
        self.interval = interval
        self.counter = 0

    def calculateMedian(self, frames):
        self.median_img = np.median(frames, axis=0).astype(dtype=np.uint8) 
        self.median_gray = cv.cvtColor(self.median_img, cv.COLOR_BGR2GRAY)
        
    # N=cantidad de frames para mediana, interval= seconds, frame actual   
    def apply(self, frame):
        if len(self.last_frames) == 0:
            # tomar N primeros del video
            print("first")
            self.calculateMedian(takeFirstNFramesMedian('vtest.avi', 10))
    
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame_foreground = cv.absdiff(frame_gray, self.median_gray)
    
        ret,foreground_mask = cv.threshold(frame_foreground, 50, 255, cv.THRESH_BINARY)
        
        self.last_frames.append(frame)
        return self.median_gray, foreground_mask
    
    def updateCounter(self):
        #print("Update median")
        self.counter += 1
        if self.counter >= int(self.interval/30 * 1000):
            # UPDATE modelo
            self.calculateMedian(self.last_frames[-self.N:])
            self.counter = 0


In [42]:
backSub = cv.createBackgroundSubtractorMOG2()
medianSub = NaiveBackgroundSubs(100, 3) 

filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# PRIMER mediana 
# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSub.apply(frame)
    
    fbMedian, fgMedianMask = medianSub.apply(frame)

    
    cv.rectangle(fgMedianMask, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(fgMedianMask, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    cv.rectangle(fbMedian, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(fbMedian, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    cv.imshow('Background Median', fbMedian)
    cv.imshow('FG Median Mask', fgMedianMask)
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    medianSub.updateCounter()
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

first


## comparacion

* Presenta menos ruido del tipo de arena.
* Al iniciar como se toma un background previo del video, me queda la mediana con personas paradas.
* Despues de un tiempo, se estabiliza y detecta regularmente bien las personas caminando.
* De vez en cuando se toman personas como parte del background, debido a que por momentos pueden estar sin moverse.
